## Relación entre convoluciones y suma de los elementos más cercanos de una matriz

Explicación paso a paso de la función 
```Python
def Hamiltonian(J, spin, H = 0): 
    kernel = sp.ndimage.generate_binary_structure(2,1)
    kernel[1][1] = False
    E_0 = spin*sp.ndimage.convolve(spin, kernel, mode= 'wrap', cval=0)
    
    return -H*spin.sum() -J*E_0.sum()
```

### Argumentos 
* `J`: intensidad de acoplamiento entre spines
* `spin`: arreglo bi-dimensional con una determinada configuración para la grilla NxN
* `H`: intensidad de campo magnético, es opcional, si no se pasa el valor se asume que es 0

### Funcionalidad

Se tienen las siguientes funciones

```Python
sp.ndimage.generate_binary_structure(rango,conectividad)
```

#### Argumentos
* `rango`: valor entero que determina el rango de la estructura. *ojo*: es el rango, por ejemplo rango 2 implica arreglo bidimensional 3x3, rango 3 implica arreglo tridimensional 3x3x3.
* `conectividad`: valor entero que indica si se consideran vecinos del elemento central, 1 implica que las diagonales se descartan.

```Python
scipy.ndimage.convolve(input, weight, output=None, mode, cval, origin=0)
```
#### Argumentos

* `input`: arreglo de N dimensiones 
* `weight`: arreglo de N dimensiones (igual a la entrada), su tamaño de una dimensión siempre es 3, ejemplo una entrada 2D tiene un peso de la forma 3x3
*  `output`: arreglo donde se guardan los resultados, 'None' solo hace que la función retorne un arreglo sin guardarlo.
* `mode`: determina como extender las fronteras, `wrap`envuelve el arreglo y `constant` rellena extiende por una constante dada.
* `cval`: constante por la que se extiende en el modo `constant`, si en nuestro caso considerasemos las froteras fijas en la suma de elementos más cercanos, los valores en las frontera tendrían un `cval=0`.
* `origin`: donde se centra la convolución, si es 0, se centra en el elemento que corresponde al centro de la matriz 3x3 al rededor de un punto. 

Aquí dejo la documentación
https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve.html#scipy.ndimage.convolve

In [36]:
import scipy as sp
import numpy as np

In [37]:
kernel = sp.ndimage.generate_binary_structure(2,1)
kernel

array([[False,  True, False],
       [ True,  True,  True],
       [False,  True, False]])

Al operar con números el kernel es el equivalente a la siguiente matriz

In [38]:
1*kernel

array([[0, 1, 0],
       [1, 1, 1],
       [0, 1, 0]])

Al hacer el elemento central falso se obtine 

In [39]:
kernel[1][1] = False
1*kernel

array([[0, 1, 0],
       [1, 0, 1],
       [0, 1, 0]])

In [40]:
np.random.seed(10) #hace que el resultado de random sea reproducible
arreglo_prueba = np.random.randint(10,size=(5,5), dtype='int')
arreglo_prueba

array([[9, 4, 0, 1, 9],
       [0, 1, 8, 9, 0],
       [8, 6, 4, 3, 0],
       [4, 6, 8, 1, 8],
       [4, 1, 3, 6, 5]])

tomemos el elemento `array[2][2]`= 4 y hagamos una matriz 3x3 con sus elementos más cercanos(incluyendo diagonales)

$$
\begin{pmatrix}
    1 & 8 & 9 \\
    6 & 4 & 3 \\
    6 & 8 & 1 \\ 
\end{pmatrix}
$$

Hay dos funciones en scipy, está convolve y correlate, convolve hace que el kernel se invierta al pesar el arreglo, ejemplo, sea el kernel 

$$
\begin{pmatrix}
    1 & 1 & 1 \\
    1 & 1 & 0 \\
    1 & 0 & 0 \\ 
\end{pmatrix}
$$

Lo que hace convolve es multiplicar la matriz de elementos cercanos con el peso invertido de la siguiente manera

##### Pesos aplicados del convolve
$$
\begin{pmatrix}
    0 & 0 & 1 \\
    0 & 1 & 1 \\
    1 & 1 & 1 \\ 
\end{pmatrix}
$$

$$
\begin{pmatrix}
    1*0 & 8*0 & 9*1 \\
    6*0 & 4*1 & 3*1 \\
    6*1 & 8*1 & 1*1 \\ 
\end{pmatrix}
$$

Esto resulta en 

\begin{pmatrix}
    0 & 0 & 9 \\
    0 & 4 & 3 \\
    6 & 8 & 1 \\ 
\end{pmatrix}
$$

Luego se suman todos los elementos de la matriz similar a un np.array.sum

Entonces la nueva entrada `array[2][2]` = 0 + 0 + 9 + 0 + 4 + 3 + 6 + 8 + 1 = 31



In [41]:
kernel1 = np.array([[1,1,1],[1,1,0],[1,0,0]])
arreglo = sp.ndimage.convolve(arreglo_prueba, kernel1, mode= 'wrap')
arreglo[2][2]

31

Para la suma de elementos cercanos usamos el kernel planteado arriba lo que nos da para el mismo arreglo de prueba

**Kernel invertido = kernel original en este caso** 
$$
\begin{pmatrix}
    0 & 1 & 0 \\
    1 & 0 & 1 \\
    0 & 1 & 0 \\ 
\end{pmatrix}
$$

$$
\begin{pmatrix}
    1*0 & 8*1 & 9*0 \\
    6*1 & 4*0 & 3*1 \\
    6*0 & 8*1 & 1*0 \\ 
\end{pmatrix}
$$

Entonces la nueva entrada `array[2][2]` = 0 + 8 + 0 +
                                          
                                   6 + 0 + 3 +
                                          
                                   0 + 8 + 0   = 25
                                   
y es la suma de los elementos más cercanos.

In [47]:
arreglo = sp.ndimage.convolve(arreglo_prueba, kernel, mode= 'wrap') #usa el kernel que usamos en nuestro código
arreglo[2][2]

25

Como el kernel es simétrico otro método es usar sp.ndimage.correlate que tiene los mismo argumentos y la misma función que covolve, pero no invierte el kernel.

Veamos el elemento centrado en el indice `array[1][1]`

$$
\begin{pmatrix}
    9 & 4 & 0 \\
    0 & 1 & 8 \\
    8 & 6 & 4 \\ 
\end{pmatrix}
$$

La suma de los elementos más cercanos es 

suma = 4 + 0 + 8 + 6 = 18 

In [50]:
arreglo1 = sp.ndimage.correlate(arreglo_prueba, kernel, mode = 'wrap')
arreglo[1][1]

18

La pregunta es, para el caso en el que necesitamos esto ¿hay alguna diferencia la velocidad?

Midiendo el tiempo en que tardan en ejecutarse con el comando mágico `%timeit function()`

In [53]:
def correlate(array, kern):
    return sp.ndimage.correlate(array, kern, mode = 'wrap')

def convolve(array, kern):
    return sp.ndimage.correlate(array, kern, mode = 'wrap')

In [62]:
%timeit correlate(arreglo_prueba, kernel)

15.9 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [63]:
%timeit convolve(arreglo_prueba, kernel)

11.1 µs ± 532 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Como se ve son muy similares, pero lo corrí varias veces y parece que convolve es ligeramente más rápido como unos 3 $\mu s$, para iteraciones muy grande nos conviene convolve.

Por último para el hamiltoniano, lo único que hay que hacer es multiplicar elemento por elemento el arreglo de spines por el del resultado de convolve esto da en el i-ésimo indice

$$
\begin{equation}
    E_{0,i,j} = spin\cdot array_{i,j} = \sigma_{i,j}\cdot(\sigma_{i+1,j} + \sigma_{i-1,j} + \sigma_{i,j+1} + \sigma_{i,j-1})
\end{equation}
$$

después para obtener la suma total de eso basta un `-J*E_0.sum()`, esto nos da con $H = 0$

$$
\begin{equation}
    \mathscr{H} = -J\sum_{<i,j>}^N{\sigma_{i,j}\cdot(\sigma_{i+1,j} + \sigma_{i-1,j} + \sigma_{i,j+1} + \sigma_{i,j-1})}
\end{equation}
$$

Aquí está la equivalencia que les comenté y es mucho más rápido que con los for loops.